#### Génération du graph "communes" dans Neo4J

Notebook amorcé par : Jesshuan

Objectifs :

- Récupération du graph communes avec points d'intérêt touristiques sous forme edges et nodes depuis les notebooks de mika... (fichier finalement transféré à l'a'ncienne)

- jointure avec infos complémentaires sur les communes (nom, etc...)

- jointure avec infos complémentaires sur les immatriculations de véhicules

- Génération dans une base de données locale Neo4j

Imports :

In [1]:
import geopandas as gpd
import pandas as pd
import geopy.distance
import os
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import copy
import joblib

from neo4j import GraphDatabase, basic_auth

In [2]:
PASSWORD_NEO4J = 'passwordneo4j'

--------

In [4]:
class CFG:
    area = [
        'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]
    area_name = 'france'      #'_'.join(area)

    

    bornes = False

    viz = False

    roads = (
        '|motorway'
        '|motorway_link'
        '|trunk'
        '|trunk_link'
        '|primary'
        '|primary_link'
        '|secondary'
        '|secondary_link'
        '|tertiary'
         '|tertiary_link'
    )

        # '|primary'
        # '|motorway_link'
        
        # '|primary_link'

        # '|secondary'
        # '|secondary_link'
        
        
        #  '|tertiary'
        #  '|tertiary_link'

    data_path = './../data_osm_communes/'

import joblib
joblib.dump(CFG.area, 'CFG.pkl')    
import os
os.environ['USE_PYGEOS'] = '0'
%load_ext autoreload
%autoreload



%aimport OSM_helper_functions
%aimport OCM_helper_functions

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
from OSM_helper_functions import *
from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Brittany': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    # 'Corse-du-Sud': '2A',
    # 'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
AREA

['Ain',
 'Allier',
 'Ardèche',
 'Cantal',
 'Drôme',
 'Isère',
 'Loire',
 'Haute-Loire',
 'Puy-de-Dôme',
 'Rhône',
 'Savoie',
 'Haute-Savoie',
 "Côte-d'Or",
 'Doubs',
 'Jura',
 'Nièvre',
 'Haute-Saône',
 'Saône-et-Loire',
 'Yonne',
 'Territoire de Belfort',
 "Côtes-d'Armor",
 'Finistère',
 'Ille-et-Vilaine',
 'Morbihan',
 'Cher',
 'Eure-et-Loir',
 'Indre',
 'Indre-et-Loire',
 'Loir-et-Cher',
 'Loiret',
 'Ardennes',
 'Aube',
 'Marne',
 'Haute-Marne',
 'Meurthe-et-Moselle',
 'Meuse',
 'Moselle',
 'Bas-Rhin',
 'Haut-Rhin',
 'Vosges',
 'Aisne',
 'Nord',
 'Oise',
 'Pas-de-Calais',
 'Somme',
 'Paris',
 'Seine-et-Marne',
 'Yvelines',
 'Essonne',
 'Hauts-de-Seine',
 'Seine-Saint-Denis',
 'Val-de-Marne',
 "Val-d'Oise",
 'Calvados',
 'Eure',
 'Manche',
 'Orne',
 'Seine-Maritime',
 'Charente',
 'Charente-Maritime',
 'Corrèze',
 'Creuse',
 'Dordogne',
 'Gironde',
 'Landes',
 'Lot-et-Garonne',
 'Pyrénées-Atlantiques',
 'Deux-Sèvres',
 'Vienne',
 'Haute-Vienne',
 'Ariège',
 'Aude',
 'Aveyron',
 'Gard

In [5]:
nodes = gpd.read_feather(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes_communes.feather'))

In [6]:
nodes_reset = nodes.reset_index()

In [7]:
nodes_reset

,osmid,y,x,codgeo,area,sleeping,visit,working,fooding,vehicule,geometry
0,1149451,50.582893,2.143174,62053,sw}KqgzsHtFrFwWhPzQtg@vn@lj@vd@jVx^p_@deAzeA`I...,3.0,1.0,NaN,NaN,NaN,POINT (2.14317 50.58289)
1,910704,49.717236,3.012390,80284,wwsQaxhnHqLqv@|Aea@`]md@zGLr`@k\hXiJvCaOcCs|@l...,NaN,1.0,NaN,NaN,NaN,POINT (3.01239 49.71724)
2,1452239,42.876728,0.610023,31290,scwB_ptdGiNs]tPyd@|e@aEt~@zEvrAdBp`@YnRbMuF|W}...,NaN,NaN,NaN,NaN,NaN,POINT (0.61002 42.87673)
3,2173628,48.094594,1.514000,28400,grkHusqdHvZcW`]_^z_AbWpLvXfQtDlK}Mnf@|O{GfL|Nv...,NaN,NaN,NaN,NaN,NaN,POINT (1.51400 48.09459)
4,106436,47.193507,0.099943,37220,oaZqd~~GntAaKgQoh@zGgFthAwXl`Aml@lv@iRpLnOgGzM...,NaN,2.0,NaN,NaN,NaN,POINT (0.09994 47.19351)
...,...,...,...,...,...,...,...,...,...,...,...
34455,2893288,46.506657,3.092677,03170,kpyQaka{Gbt@lCvJyZn\}W~KfFvWfe@}Ad_@sSlf@qp@l`...,NaN,2.0,2.0,NaN,NaN,POINT (3.09268 46.50666)
34456,186959,49.015889,-0.162877,14394,hhe@mmfjHtCxd@mmA~a@ie@bSkfAbr@oOnMwhAzl@u~@im...,4.0,NaN,NaN,4.0,NaN,POINT (-0.16288 49.01589)
34457,1405692,48.302289,-3.895241,29115,zy~Vu}weHqLnp@kXd]gGtHeWp~@a]e@at@`Kyx@hPm_@cY...,NaN,1.0,NaN,NaN,NaN,POINT (-3.89524 48.30229)
34458,366442,47.370808,-0.496845,49308,n|}Aoea`Hud@uJcPoQw@kVlR{Qda@aK`t@oj@~l@{Itj@g...,5.0,4.0,NaN,1.0,NaN,POINT (-0.49684 47.37081)


Cleaning :

In [8]:
nodes_reset.drop(['area', 'geometry'], axis=1, inplace=True)

Jointure with others features for communes :

In [10]:
communes_infos = pd.read_feather('./datas/communes.feather')

In [11]:
communes_infos

,insee,nom_commune,surf_ha,x,y
0,2B222,Pie-d'Orezza,573.0,9.338151,42.374290
1,2B137,Lano,824.0,9.235358,42.378872
2,2B051,Cambia,833.0,9.302108,42.368750
3,2B106,Érone,393.0,9.266614,42.375565
4,2B185,Oletta,2674.0,9.333845,42.641773
...,...,...,...,...,...
34950,62885,Westrehem,299.0,2.343461,50.544903
34951,62795,Sibiville,738.0,2.331172,50.304585
34952,62187,Buneville,384.0,2.347346,50.324764
34953,62694,Rebreuve-sur-Canche,842.0,2.340987,50.266950


In [13]:
communes_infos.rename(columns={'x':'x_centroid','y':'y_centroid'}, inplace=True)

Jointure à gauche (pour enlever ce qui n'est pas métropolitain) :

In [15]:
df_join = nodes_reset.merge(communes_infos, how='left', left_on='codgeo', right_on='insee')

In [16]:
df_join

,osmid,y,x,codgeo,sleeping,visit,working,fooding,vehicule,insee,nom_commune,surf_ha,x_centroid,y_centroid
0,1149451,50.582893,2.143174,62053,3.0,1.0,NaN,NaN,NaN,62053,Audincthun,1520.0,2.127048,50.578743
1,910704,49.717236,3.012390,80284,NaN,1.0,NaN,NaN,NaN,80284,Esmery-Hallon,1888.0,3.024816,49.711567
2,1452239,42.876728,0.610023,31290,NaN,NaN,NaN,NaN,NaN,31290,Lège,289.0,0.598100,42.879140
3,2173628,48.094594,1.514000,28400,NaN,NaN,NaN,NaN,NaN,28400,Varize,1387.0,1.525235,48.082363
4,106436,47.193507,0.099943,37220,NaN,2.0,NaN,NaN,NaN,37220,Saint-Germain-sur-Vienne,1338.0,0.110709,47.180550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34455,2893288,46.506657,3.092677,03170,NaN,2.0,2.0,NaN,NaN,03170,Meillers,2361.0,3.101765,46.513610
34456,186959,49.015889,-0.162877,14394,4.0,NaN,NaN,4.0,NaN,14394,Maizières,718.0,-0.166225,49.013660
34457,1405692,48.302289,-3.895241,29115,NaN,1.0,NaN,NaN,NaN,29115,Lannédern,1246.0,-3.902896,48.289390
34458,366442,47.370808,-0.496845,49308,5.0,4.0,NaN,1.0,NaN,49308,Saint-Melaine-sur-Aubance,509.0,-0.501540,47.381695


In [17]:
immat = pd.read_feather('./datas/immat.feather')

In [18]:
immat

,codgeo,epci,libepci,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,insee,nom,surf_ha,x,y
0,01001,200069193,CC de la Dombes,12,0,929,01001,L'Abergement-Clémenciat,1567.0,4.925852,46.153732
1,01010,200070852,CC Usses et RhÃ´ne,21,0,1428,01010,Anglefort,2962.0,5.794870,45.909100
2,01011,200042935,CA Haut - Bugey AgglomÃ©ration,7,0,419,01011,Apremont,1498.0,5.658005,46.205708
3,01013,240100883,CC de la Plaine de l'Ain,0,0,172,01013,Arandas,1405.0,5.499066,45.890200
4,01019,200040350,CC Bugey Sud,0,0,27,01019,Armix,687.0,5.582709,45.854286
...,...,...,...,...,...,...,...,...,...,...,...
34955,85261,200073260,CC Sud VendÃ©e Littoral,5,0,281,85261,Sainte-Pexine,1600.0,-1.124170,46.560040
34956,85269,248500563,"CC VendÃ©e, SÃ¨vre, Autise",2,0,465,85269,Saint-Sigismond,1034.0,-0.707647,46.336662
34957,85273,200071629,CC Challans-Gois CommunautÃ©,15,0,1982,85273,Saint-Urbain,1653.0,-2.016456,46.881668
34958,85278,200071900,CC VendÃ©e Grand Littoral,32,1,1730,85278,Saint-Vincent-sur-Jard,1468.0,-1.548977,46.429520


In [19]:
immat.rename(columns={'x':'x_centroid','y':'y_centroid'}, inplace=True)

In [21]:
df_join

,osmid,y,x,codgeo,sleeping,visit,working,fooding,vehicule,insee,nom_commune,surf_ha,x_centroid,y_centroid
0,1149451,50.582893,2.143174,62053,3.0,1.0,NaN,NaN,NaN,62053,Audincthun,1520.0,2.127048,50.578743
1,910704,49.717236,3.012390,80284,NaN,1.0,NaN,NaN,NaN,80284,Esmery-Hallon,1888.0,3.024816,49.711567
2,1452239,42.876728,0.610023,31290,NaN,NaN,NaN,NaN,NaN,31290,Lège,289.0,0.598100,42.879140
3,2173628,48.094594,1.514000,28400,NaN,NaN,NaN,NaN,NaN,28400,Varize,1387.0,1.525235,48.082363
4,106436,47.193507,0.099943,37220,NaN,2.0,NaN,NaN,NaN,37220,Saint-Germain-sur-Vienne,1338.0,0.110709,47.180550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34455,2893288,46.506657,3.092677,03170,NaN,2.0,2.0,NaN,NaN,03170,Meillers,2361.0,3.101765,46.513610
34456,186959,49.015889,-0.162877,14394,4.0,NaN,NaN,4.0,NaN,14394,Maizières,718.0,-0.166225,49.013660
34457,1405692,48.302289,-3.895241,29115,NaN,1.0,NaN,NaN,NaN,29115,Lannédern,1246.0,-3.902896,48.289390
34458,366442,47.370808,-0.496845,49308,5.0,4.0,NaN,1.0,NaN,49308,Saint-Melaine-sur-Aubance,509.0,-0.501540,47.381695


In [22]:
df_join_tot = df_join.merge(immat, how='left', left_on='insee', right_on='codgeo')

In [24]:
df_join_tot[0:50]

,osmid,y,x,codgeo_x,sleeping,visit,working,fooding,vehicule,insee_x,...,epci,libepci,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,insee_y,nom,surf_ha_y,x_centroid_y,y_centroid_y
0,1149451,50.582893,2.143174,62053,3.0,1.0,NaN,NaN,NaN,62053,...,200069037,CA du Pays de Saint-Omer,7.0,0.0,787.0,62053,Audincthun,1520.0,2.127048,50.578743
1,910704,49.717236,3.012390,80284,NaN,1.0,NaN,NaN,NaN,80284,...,200070985,CC de l'Est de la Somme,10.0,0.0,931.0,80284,Esmery-Hallon,1888.0,3.024816,49.711567
2,1452239,42.876728,0.610023,31290,NaN,NaN,NaN,NaN,NaN,31290,...,200072635,CC PyrÃ©nÃ©es Haut Garonnaises,1.0,0.0,43.0,31290,Lège,289.0,0.598100,42.879140
3,2173628,48.094594,1.514000,28400,NaN,NaN,NaN,NaN,NaN,28400,...,200070159,CC CÅur de Beauce,2.0,0.0,228.0,28400,Varize,1387.0,1.525235,48.082363
4,106436,47.193507,0.099943,37220,NaN,2.0,NaN,NaN,NaN,37220,...,200043081,"CC Chinon, Vienne et Loire",3.0,0.0,408.0,37220,Saint-Germain-sur-Vienne,1338.0,0.110709,47.180550
5,919361,46.120200,4.220290,42333,NaN,NaN,NaN,NaN,NaN,42333,...,200035202,CC Charlieu-Belmont,5.0,0.0,674.0,42333,Villers,585.0,4.226531,46.125072
6,1453577,47.349610,6.806296,25393,1.0,1.0,NaN,NaN,NaN,25393,...,200023075,CC du Pays de MaÃ®che,4.0,0.0,796.0,25393,Montécheroux,1324.0,6.798250,47.348766
7,75922,43.863485,-0.702268,40180,1.0,2.0,1.0,NaN,NaN,40180,...,244000766,CC du Pays Tarusate,21.0,0.0,1400.0,40180,Meilhan,3915.0,-0.695854,43.859810
8,135900,43.901773,1.496646,82188,NaN,NaN,NaN,NaN,NaN,82188,...,200066652,CC Grand Sud Tarn et Garonne,16.0,0.0,695.0,82188,Varennes,1467.0,1.490033,43.916515
9,1860843,48.582169,5.088382,52347,NaN,NaN,NaN,NaN,NaN,52347,...,200068666,CA de Saint-Dizier Der et Blaise,0.0,0.0,246.0,52347,Narcy,1123.0,5.098846,48.580364


In [27]:
df_join_tot.loc[df_join_tot['insee_x']!=df_join_tot['codgeo_x'],:]

,osmid,y,x,codgeo_x,sleeping,visit,working,fooding,vehicule,insee_x,...,epci,libepci,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,insee_y,nom,surf_ha_y,x_centroid_y,y_centroid_y


In [26]:
df_join_tot.loc[df_join_tot['codgeo_x']!=df_join_tot['codgeo_y'],:]

,osmid,y,x,codgeo_x,sleeping,visit,working,fooding,vehicule,insee_x,...,epci,libepci,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,insee_y,nom,surf_ha_y,x_centroid_y,y_centroid_y
1802,2116433,49.236510,5.467500,55050,NaN,1.0,NaN,NaN,NaN,55050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11433,2116434,49.238106,5.397626,55307,NaN,1.0,NaN,NaN,NaN,55307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19057,2967703,49.233925,5.265244,55139,NaN,1.0,NaN,NaN,NaN,55139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23990,280440,49.181231,-0.221640,14666,NaN,NaN,NaN,NaN,NaN,14666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25503,6837719,49.213400,1.330809,27058,2.0,1.0,NaN,NaN,NaN,27058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_join_tot.loc[df_join_tot['surf_ha_x']!=df_join_tot['surf_ha_y'],:]

,osmid,y,x,codgeo_x,sleeping,visit,working,fooding,vehicule,insee_x,...,epci,libepci,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,insee_y,nom,surf_ha_y,x_centroid_y,y_centroid_y
1802,2116433,49.236510,5.467500,55050,NaN,1.0,NaN,NaN,NaN,55050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11433,2116434,49.238106,5.397626,55307,NaN,1.0,NaN,NaN,NaN,55307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19057,2967703,49.233925,5.265244,55139,NaN,1.0,NaN,NaN,NaN,55139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23990,280440,49.181231,-0.221640,14666,NaN,NaN,NaN,NaN,NaN,14666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25503,6837719,49.213400,1.330809,27058,2.0,1.0,NaN,NaN,NaN,27058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_join_tot.loc[df_join_tot['x_centroid_x']!=df_join_tot['x_centroid_y'],:]

,osmid,y,x,codgeo_x,sleeping,visit,working,fooding,vehicule,insee_x,...,epci,libepci,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,insee_y,nom,surf_ha_y,x_centroid_y,y_centroid_y
1802,2116433,49.236510,5.467500,55050,NaN,1.0,NaN,NaN,NaN,55050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11433,2116434,49.238106,5.397626,55307,NaN,1.0,NaN,NaN,NaN,55307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19057,2967703,49.233925,5.265244,55139,NaN,1.0,NaN,NaN,NaN,55139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23990,280440,49.181231,-0.221640,14666,NaN,NaN,NaN,NaN,NaN,14666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25503,6837719,49.213400,1.330809,27058,2.0,1.0,NaN,NaN,NaN,27058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
columns_to_drop = ['insee_y','codgeo_y', 'codgeo_x', 'surf_ha_y', 'x_centroid_y', 'y_centroid_y']

In [32]:
df_join_tot.drop(columns_to_drop, axis=1, inplace=True)

In [33]:
df_join_tot

,osmid,y,x,sleeping,visit,working,fooding,vehicule,insee_x,nom_commune,surf_ha_x,x_centroid_x,y_centroid_x,epci,libepci,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,nom
0,1149451,50.582893,2.143174,3.0,1.0,NaN,NaN,NaN,62053,Audincthun,1520.0,2.127048,50.578743,200069037,CA du Pays de Saint-Omer,7.0,0.0,787.0,Audincthun
1,910704,49.717236,3.012390,NaN,1.0,NaN,NaN,NaN,80284,Esmery-Hallon,1888.0,3.024816,49.711567,200070985,CC de l'Est de la Somme,10.0,0.0,931.0,Esmery-Hallon
2,1452239,42.876728,0.610023,NaN,NaN,NaN,NaN,NaN,31290,Lège,289.0,0.598100,42.879140,200072635,CC PyrÃ©nÃ©es Haut Garonnaises,1.0,0.0,43.0,Lège
3,2173628,48.094594,1.514000,NaN,NaN,NaN,NaN,NaN,28400,Varize,1387.0,1.525235,48.082363,200070159,CC CÅur de Beauce,2.0,0.0,228.0,Varize
4,106436,47.193507,0.099943,NaN,2.0,NaN,NaN,NaN,37220,Saint-Germain-sur-Vienne,1338.0,0.110709,47.180550,200043081,"CC Chinon, Vienne et Loire",3.0,0.0,408.0,Saint-Germain-sur-Vienne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34455,2893288,46.506657,3.092677,NaN,2.0,2.0,NaN,NaN,03170,Meillers,2361.0,3.101765,46.513610,200071496,CC du Bocage Bourbonnais,0.0,0.0,172.0,Meillers
34456,186959,49.015889,-0.162877,4.0,NaN,NaN,4.0,NaN,14394,Maizières,718.0,-0.166225,49.013660,241400514,CC du Pays de Falaise,8.0,0.0,595.0,Maizières
34457,1405692,48.302289,-3.895241,NaN,1.0,NaN,NaN,NaN,29115,Lannédern,1246.0,-3.902896,48.289390,200067247,CC Pleyben-ChÃ¢teaulin-Porzay,1.0,0.0,340.0,Lannédern
34458,366442,47.370808,-0.496845,5.0,4.0,NaN,1.0,NaN,49308,Saint-Melaine-sur-Aubance,509.0,-0.501540,47.381695,200071553,CC Loire Layon Aubance,57.0,0.0,2388.0,Saint-Melaine-sur-Aubance


In [34]:
df_join_tot.loc[df_join_tot['nom_commune']!=df_join_tot['nom'],:]

,osmid,y,x,sleeping,visit,working,fooding,vehicule,insee_x,nom_commune,surf_ha_x,x_centroid_x,y_centroid_x,epci,libepci,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,nom
1802,2116433,49.236510,5.467500,NaN,1.0,NaN,NaN,NaN,55050,Bezonvaux,926.0,5.471458,49.231598,NaN,NaN,NaN,NaN,NaN,NaN
11433,2116434,49.238106,5.397626,NaN,1.0,NaN,NaN,NaN,55307,Louvemont-Côte-du-Poivre,831.0,5.401587,49.240210,NaN,NaN,NaN,NaN,NaN,NaN
19057,2967703,49.233925,5.265244,NaN,1.0,NaN,NaN,NaN,55139,Cumières-le-Mort-Homme,611.0,5.268401,49.234490,NaN,NaN,NaN,NaN,NaN,NaN
23990,280440,49.181231,-0.221640,NaN,NaN,NaN,NaN,NaN,14666,Sannerville,520.0,-0.224219,49.183563,NaN,NaN,NaN,NaN,NaN,NaN
25503,6837719,49.213400,1.330809,2.0,1.0,NaN,NaN,NaN,27058,Les Trois Lacs,3657.0,1.333144,49.216904,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_join_tot.drop(['nom'], axis=1, inplace=True)

In [37]:
df_join_tot

,osmid,y,x,sleeping,visit,working,fooding,vehicule,insee_x,nom_commune,surf_ha_x,x_centroid_x,y_centroid_x,epci,libepci,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp
0,1149451,50.582893,2.143174,3.0,1.0,NaN,NaN,NaN,62053,Audincthun,1520.0,2.127048,50.578743,200069037,CA du Pays de Saint-Omer,7.0,0.0,787.0
1,910704,49.717236,3.012390,NaN,1.0,NaN,NaN,NaN,80284,Esmery-Hallon,1888.0,3.024816,49.711567,200070985,CC de l'Est de la Somme,10.0,0.0,931.0
2,1452239,42.876728,0.610023,NaN,NaN,NaN,NaN,NaN,31290,Lège,289.0,0.598100,42.879140,200072635,CC PyrÃ©nÃ©es Haut Garonnaises,1.0,0.0,43.0
3,2173628,48.094594,1.514000,NaN,NaN,NaN,NaN,NaN,28400,Varize,1387.0,1.525235,48.082363,200070159,CC CÅur de Beauce,2.0,0.0,228.0
4,106436,47.193507,0.099943,NaN,2.0,NaN,NaN,NaN,37220,Saint-Germain-sur-Vienne,1338.0,0.110709,47.180550,200043081,"CC Chinon, Vienne et Loire",3.0,0.0,408.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34455,2893288,46.506657,3.092677,NaN,2.0,2.0,NaN,NaN,03170,Meillers,2361.0,3.101765,46.513610,200071496,CC du Bocage Bourbonnais,0.0,0.0,172.0
34456,186959,49.015889,-0.162877,4.0,NaN,NaN,4.0,NaN,14394,Maizières,718.0,-0.166225,49.013660,241400514,CC du Pays de Falaise,8.0,0.0,595.0
34457,1405692,48.302289,-3.895241,NaN,1.0,NaN,NaN,NaN,29115,Lannédern,1246.0,-3.902896,48.289390,200067247,CC Pleyben-ChÃ¢teaulin-Porzay,1.0,0.0,340.0
34458,366442,47.370808,-0.496845,5.0,4.0,NaN,1.0,NaN,49308,Saint-Melaine-sur-Aubance,509.0,-0.501540,47.381695,200071553,CC Loire Layon Aubance,57.0,0.0,2388.0


In [39]:
df_join_tot.rename(columns={'insee_x':'insee', 'surf_ha_x':'surf_ha','x_centroid_x':'x_centroid', 'y_centroid_x':'y_centroid'}, inplace=True)

In [42]:
df_join_tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34460 entries, 0 to 34459
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   osmid                    34460 non-null  object 
 1   y                        34460 non-null  float64
 2   x                        34460 non-null  float64
 3   sleeping                 15412 non-null  float64
 4   visit                    17921 non-null  float64
 5   working                  6950 non-null   float64
 6   fooding                  6616 non-null   float64
 7   vehicule                 4166 non-null   float64
 8   insee                    34460 non-null  object 
 9   nom_commune              34460 non-null  object 
 10  surf_ha                  34460 non-null  float64
 11  x_centroid               34460 non-null  float64
 12  y_centroid               34460 non-null  float64
 13  epci                     34455 non-null  object 
 14  libepci               

--------------------------

Fonctions de requêtes neo4j :

In [5]:
def delete_all(tx):
    query = 'MATCH (n) \
                DETACH DELETE n \
                RETURN count(*) AS COUNT'
    result = tx.run(query)
    return result.data()

In [22]:
def delete_city(tx):
    query = 'MATCH (n:CITY) \
                DETACH DELETE n \
                RETURN count(*) AS COUNT'
    result = tx.run(query)
    return result.data()

In [56]:
def delete_just_edges(tx):
    query = 'MATCH ()-[r]-() \
                DELETE r \
                RETURN count(*) AS COUNT'
    result = tx.run(query)
    return result.data()

In [6]:
def create_node(tx, node):
    query = "   WITH $node AS node \
                CREATE (r:ROAD {road_index:node}) \
                RETURN count(r) AS COUNT"
    
    result = tx.run(query, node=node)
    return result.data()

In [47]:
def create_node_per_batch(tx, props_list):

    query = "   UNWIND $props_list AS map \
                CREATE (n:CITY) \
                SET n = map \
                RETURN count(n) AS COUNT"

    result = tx.run(query, props_list=props_list)
    return result.data()

In [8]:
def create_edge_between_ROAD_POINTS(tx, props_list):

    query = "   UNWIND $props_list AS map \
                MERGE (n1:ROAD_POINT {osmid : map.u})-[r:CONNECT]->(n2:ROAD_POINT {osmid : map.v}) \
                SET r = map \
                RETURN count(r) AS COUNT"

    result = tx.run(query, props_list=props_list)
    return result.data()

In [9]:
def check_collection(tx, id):

    query = "  WITH $id AS id \
                MATCH (n) \
                WHERE n.osmid = id \
                RETURN count(n) \
                 "
    
    result = tx.run(query, id=id)
    return result.data()

In [10]:
def all_fusion_nodes(tx, id):

    query = "   WITH $id AS id \
                MATCH (n) \
                WHERE n.osmid = id \
                WITH collect(n) AS nodes \
                CALL apoc.refactor.mergeNodes( \
                                        nodes, \
                                        {properties: { \
                                        osmid: 'discard', \
                                        `.*`:'combine'} \
                                            }) \
                YIELD node \
                RETURN node "
    
    result = tx.run(query,  id=id)
    return result.data()

In [11]:
def match_alone_nodes(tx):
    query = " MATCH (r) \
            WHERE NOT (r)-[:LINKED_TO]-() \
            RETURN r.road_index AS road_index"
    
    result = tx.run(query)
    return result.data()

Drivers session for Neo4j :

In [43]:
driver = GraphDatabase.driver(
  "bolt://localhost:7687",
  auth=basic_auth("neo4j", PASSWORD_NEO4J))

Ardoise (pour effacer tout ancien graph)

In [23]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(delete_city)

driver.close()

nb_nodes = result[0]['COUNT']
                
print(f'Total Numbers of deleted nodes: {nb_nodes}')

Total Numbers of deleted nodes: 34460


In [40]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(delete_all)

driver.close()

nb_nodes = result[0]['COUNT']
                
print(f'Total Numbers of deleted nodes: {nb_nodes}')

Total Numbers of deleted nodes: 2


Ardoise (pour effacer seulement les relations)

In [158]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(delete_just_edges)

driver.close()

nb_edges = result[0]['COUNT']
                
print(f'Total Numbers of deleted edges: {nb_edges}')

Total Numbers of deleted edges: 0


---

NODES GENERATION :

In [44]:
print("conversion of objects to parameters list for neo4j...")

props_list = df_join_tot.to_dict("records")

conversion of objects to parameters list for neo4j...


In [45]:
BATCH_SIZE = 10000

In [46]:
# --- Batch function ---

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]


In [48]:
with driver.session() as session:

    count_list = []

    # Batch the writing session :

    for props_batch in batch(props_list, BATCH_SIZE):
        result = session.write_transaction(create_node_per_batch, props_batch)
        count_list.append(result[0]['COUNT'])
        nb_updates = sum(count_list)
        print(f'Number of nodes created : {nb_updates}...')

        

    driver.close()

print(f'Bath_list: {count_list}')

print(f'Total Numbers of nodes created/updates: {nb_updates}')

/tmp/ipykernel_23099/3878216737.py:8: DeprecationWarning: write_transaction has been renamed to execute_write
  result = session.write_transaction(create_node_per_batch, props_batch)


Number of nodes created : 10000...
Number of nodes created : 20000...
Number of nodes created : 30000...
Number of nodes created : 34460...
Bath_list: [10000, 10000, 10000, 4460]
Total Numbers of nodes created/updates: 34460


---